# Model2
This notebook is a deeper iteration of similar architecture from model 1 set up for the purpose of running on AWS.

In [10]:
import os
import sagemaker
import tensorflow as tf
from sagemaker.tensorflow import TensorFlow

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [2]:
bucket = 'sagemaker-200928-21h35m'
pkl_dir = f's3://{bucket}/PKL'

In [ ]:
# copying file

desc_path        = os.path.join(pkl_dir, 'condensed_descriptions.pkl')
feat_path        = os.path.join(pkl_dir, 'full_features.pkl')
train_list_path  = os.path.join(pkl_dir, 'train_list_full.pkl')
val_list_path    = os.path.join(pkl_dir, 'val_list_full.pkl')
test_list_path   = os.path.join(pkl_dir, 'test_list.pkl')
test_art_path    = os.path.join(pkl_dir, 'test_list_art.pkl')

!aws --region {region} s3 cp {desc_path}
!aws --region {region} s3 cp {feat_path}
!aws --region {region} s3 cp {train_list_path}
!aws --region {region} s3 cp {val_list_path}
!aws --region {region} s3 cp {test_list_path}
!aws --region {region} s3 cp {test_art_path}

descriptions = pickle.load('condensed_descriptions.pkl')
features = pickle.load('full_features.pkl')
train_list_full = pickle.load('train_list_full.pkl')
val_list_full = pickle.load('val_list_full.pkl')
test_list = pickle.load('test_list.pkl')
test_list_art = pickle.load('test_list_art.pkl')


In [ ]:
from SCRIPT.sequence_generator import *

In [ ]:
processor = sequence_generator(descriptions, features)

train_X1, train_X2, train_Y = processor.train_generator(train_list_full)
val_X1, val_X2, val_Y = processor.validation_generator(val_list_full)

# get params
tokenizer = processor.get_tokenizer()
max_length = processor.get_max_length()
num_vocab = processor.get_num_vocab()

## Local Mode

In [5]:
batch_size = 128
epochs = 1

In [6]:
tf_estimator = TensorFlow(entry_point = 'SCRIPT/nlg_model_tf_1.py', 
                         role = role, 
                         train_instance_count = 1, 
                         train_instance_type = 'local', 
                         py_version = 'py37', 
                         framework_version = '1.15.2',
                         script_mode = True, 
                         hyperparameters = {'epochs': epochs, 
                                           'batch-size': batch_size}
                         )

In [7]:
tf_estimator.fit({'pkl': pkl_dir})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Creating tmp9e2lguv4_algo-1-nfqu5_1 ... 
Attaching to tmp9e2lguv4_algo-1-nfqu5_12mdone
algo-1-nfqu5_1  | WARNING:tensorflow:From /usr/local/lib/python3.7/site-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
algo-1-nfqu5_1  | 
algo-1-nfqu5_1  | 2020-09-29 02:39:27,697 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-nfqu5_1  | 2020-09-29 02:39:27,703 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-nfqu5_1  | 2020-09-29 02:39:28,877 sagemaker-training-toolkit INFO     Module nlg_model_tf_1.py does not provide a setup.py. 
algo-1-nfqu5_1  | Generating setup.py
algo-1-nfqu5_1  | 2020-09-29 02:39:28,878 sagemaker-training-toolkit INFO     Generating setup.cfg
algo-1-nfqu5_1  | 2020-09-29 02:39:28,878 sagemaker-training-toolkit INFO     Generating MANIFEST.in
algo-1-nfqu5_1  | 2020-09-29 02:39:28,878 sagemaker-

RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmp9e2lguv4/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1

In [ ]:
tf_estimator = TensorFlow(entry_point = 'SCRIPT/nlg_model_tf_1.py', 
                         role = role, 
                         train_instance_count = 1, 
                         train_instance_type = 'ml.p2.xlarge',
                         py_version = 'py37', 
                         framework_version = '1.15.2',
                         script_mode = True, 
                         hyperparameters = {'epochs': epochs, 
                                           'batch-size': batch_size}
                         )

In [ ]:
tf_estimator.fit({'pkl': pkl_dir})

## Deploy

In [ ]:
import time

tf_endpoint = 'nlg-model-tf-'+time.strftime('%Y-%m-%d-%H-%M-%S', time.gmtime())

# GPU
#tf_predictor = tf_estimator.deploy(initial_instance_count = 1, 
#                                   instance_type = 'ml.p2.xlarge')


tf_predictor = tf_estimator.deploy(initial_instance_count = 1, 
                                  instance_type = 'ml.c5.large', 
                                  accelerator_type = 'ml.eia1.medium', 
                                  endpoint_name = tf_endpoint)

## Evaluation

In [ ]:
from SCRIPT.evaluation_tools import *

In [ ]:
# testing on photo data
pred_list, score = get_bleu(test_list, 
                            features, 
                            tokenizer, 
                            max_length, 
                            tf_predictor, 
                            descriptions)

In [ ]:
# testing on photo data
artpred_list, artscore = get_bleu(test_list_art, 
                            features, 
                            tokenizer, 
                            max_length, 
                            tf_predictor, 
                            descriptions)

In [ ]:
#flicker_descriptor = descriptor(features, tokenizer, processor, model)
#flicker_descriptor.test_random_image(img_id)

In [ ]:
#art_descriptor = descriptor(features, tokenizer, processor, model,img_dir = art_dir)
#art_descriptor.test_one_image(img_id)

## Delete End-point

In [ ]:
sess.delete_endpoint(endpoint_name = tf_endpoint)